In [1]:
from srcmodules import Objects, Robots, Tasks, Users
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

Firstly, let's go through iteration cycle
### 1. Gesture and actions setup

In [3]:
A = Actions.A
G = Gestures.G
U = Users.U
UID = Users.selected_id
print(A)
print(G)
print(U)
print(UID)

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']
['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward']
['Jan', 'Mara']
0


### 2. Generate policy
- Human Target action & Target object (**TaTo**) are the real values, which we want to determine
- Let's generate scene_state which is an input to the model
- And create_observations: `focus_point` and `gestures_vector` from TaTo
- Based on that we generate simple policy, which can be diagonal at start

In [4]:
TaTo = ('open', 'drawer')
scene_state = s.generate_scene_state(A, G, U, UID)

bn = MappingBayesNet(scene_state)
print(bn.create_observation(TaTo))
bn.init_policy_simple()

{'focus_point': array([[2.08299352, 0.03790751, 0.01821753]]), 'gesture_vec': array([0.19932437, 0.44597669, 0.10266543, 0.01188741, 0.11100262,
       0.12914349])}


{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init',
 'r': False,
 'permutation': 0}

### 3. Task
- Choose task and generate start (s) and target scenes (s2)

In [19]:
t = Tasks.CupToDrawer()
s, s2 = t.get_start_target_scenes()

In [20]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 1 0], empty
drawer, drawer, [1 1 0], closed, []
Robot: [0 0 3], opened, rotation: 0, attached: -


In [21]:
s2.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 1 0], empty
drawer, drawer, [1 1 0], opened, ['cup']
Robot: [0 0 3], opened, rotation: 0, attached: -


###  4. Select action
- Selects an action based on the observations

In [22]:
action = bn.select_action()
action

{'target_object': 'drawer', 'target_action': 'open'}

### 5. New world state
- Updates the scene


In [23]:
# temporary place
Actions.do(s, action, ignore_location=True)
s.drawer.info

drawer, drawer, [1 1 0], opened, []


### 6. Reward computation
- When reward is `True`, scenes are the same and task is done

In [24]:
s == s2

5.0

### 7. Complete the task as an action sequence

In [32]:
s, s2 = t.get_start_target_scenes()
reward = s == s2
print(f"Reward at the start: {reward}")

action_sequence = Tasks.CupToDrawer.action_sequence
for action_input in action_sequence:
    scene_state = s.generate_scene_state(A, G, U, UID)

    bn = MappingBayesNet(scene_state)
    bn.init_policy_simple()
    bn.create_observation(action_input)
    
    action = bn.select_action()
    print(action)
    
    if not Actions.do(s, action, ignore_location=True):
        raise Exception("Action cannot be executed")
    
    reward = s == s2
    print(f"Reward: {reward}")


Reward at the start: 4.0
{'target_object': 'drawer', 'target_action': 'open'}
Reward: 5.0
{'target_object': 'cup', 'target_action': 'pick_up'}
Reward: 5.0
{'target_object': 'drawer', 'target_action': 'put'}
Reward: True


In [33]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 1 0], empty
drawer, drawer, [3 1 0], opened, ['cup']
Robot: [0 0 3], opened, rotation: 0, attached: -


# Attempt 1: Loop with reward

- prototype
- written notes and suggestions
- not working

In [47]:
# Static params
A = Actions.A
G = Gestures.G
U = Users.U
UID = Users.selected_id

# hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

In [48]:
# Task
t = Tasks.CupToDrawer()
# train loops 
n = 2
for i in range(1,n+1):
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    # Init. task start scene 's', target scene 's2'
    s, s2 = t.get_start_target_scenes()
    
    # Generate first scene state
    state = s.generate_scene_state(A, G, U, UID)
    bn = MappingBayesNet(state)
    bn.init_policy_simple()
    while not done:
        # Exploring actions space x Using learned values
        if np.random.uniform(0, 1) < epsilon:
            # There should be: Sampling from env/scene (p=10%)
            '''
            action = scene.sample()
            '''
            TaTo_action = t.decide(s)
        else:
            # There should be: Pick action using learned values (p=90%)
            '''
            action = np.argmax(q_table[state])
            '''
            TaTo_action = t.decide(s)
        
        
        print(f"Action: {TaTo_action}")
        # Apply action, compute reward, check if done
        s, reward, done, step_done = Actions.step(s, s2, TaTo_action, ignore_location=True)
        # Update policy based on reward
        bn.policy_update(reward)

Action: ('open', 'drawer')
Action: ('pick_up', 'cup')
Action: ('put', 'drawer')
Action: ('open', 'drawer')
Action: ('pick_up', 'cup')
Action: ('put', 'drawer')


In [49]:
s == s2

True

In [50]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 0 0], empty
drawer, drawer, [3 2 0], opened, ['cup']
Robot: [0 0 3], opened, rotation: 0, attached: -


In [51]:
s2.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 0 0], empty
drawer, drawer, [3 2 0], opened, ['cup']
Robot: [0 0 3], opened, rotation: 0, attached: -


In [52]:
s.drawer.contains_list

['cup']

In [53]:
bn.policy['CM_est']

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

## Train the policy

In [55]:
#A = Actions.A = ['move_up', 'move_down', 'move_left', 'move_right', 'open', 'close']
policy = None
t = Tasks.CupToDrawer()

for i in range(1,101):
    epochs, penalties, reward, = 0, 0, 0
    done = False

    s, s2 = t.get_start_target_scenes()
    
    state = s.generate_scene_state(A, G, U, UID)
    bn = MappingBayesNet(state)
    bn.init_policy_simple(policy=policy)
    while not done:
        bn.create_observation(t.decide(s))
        action = bn.select_action()
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        bn.policy_update(reward)
    bn.print_policy()
    policy = bn.policy

total: 4
0, r: False 	-> init, (open, drawer)
1, r: 5.0 	-> forward, (pick_up, cup)
2, r: 5.0 	-> forward, (put, drawer)
3, r: 5.0 	-> revert, (pick_up, cup)
d: done
total: 37
0, r: 5.0 	-> revert, (close, drawer)
1, r: 4.0 	-> forward, (close, drawer)
2, r: 4.0 	-> forward, (pick_up, drawer)
3, r: 4.0 	-> forward, (pick_up, drawer)
4, r: 4.0 	-> forward, (pick_up, drawer)
5, r: 4.0 	-> forward, (move_up, drawer)
6, r: 4.0 	-> forward, (close, drawer)
7, r: 4.0 	-> forward, (open, drawer)
8, r: 5.0 	-> forward, (close, cup)
9, r: 5.0 	-> forward, (close, cup)
10, r: 5.0 	-> forward, (close, cup)
11, r: 5.0 	-> forward, (close, cup)
12, r: 5.0 	-> forward, (close, cup)
13, r: 5.0 	-> forward, (close, cup)
14, r: 5.0 	-> forward, (move_up, cup)
15, r: 5.0 	-> forward, (move_up, cup)
16, r: 5.0 	-> forward, (put, cup)
17, r: 5.0 	-> forward, (put, cup)
18, r: 5.0 	-> forward, (close, cup)
19, r: 5.0 	-> forward, (pick_up, cup)
20, r: 5.0 	-> forward, (move_up, drawer)
21, r: 5.0 	-> forwa

751, r: 5.0 	-> revert, (close, drawer)
752, r: 5.0 	-> revert, (move_up, drawer)
753, r: 5.0 	-> revert, (close, drawer)
754, r: 5.0 	-> revert, (move_up, drawer)
755, r: 5.0 	-> revert, (close, drawer)
756, r: 5.0 	-> revert, (move_up, drawer)
757, r: 5.0 	-> revert, (close, drawer)
758, r: 5.0 	-> revert, (move_up, drawer)
759, r: 5.0 	-> revert, (close, drawer)
760, r: 5.0 	-> revert, (move_up, drawer)
761, r: 5.0 	-> revert, (close, drawer)
762, r: 5.0 	-> revert, (move_up, drawer)
763, r: 5.0 	-> revert, (close, drawer)
764, r: 5.0 	-> revert, (move_up, drawer)
765, r: 5.0 	-> revert, (close, drawer)
766, r: 5.0 	-> revert, (move_up, drawer)
767, r: 5.0 	-> revert, (close, drawer)
768, r: 5.0 	-> revert, (move_up, drawer)
769, r: 5.0 	-> revert, (close, drawer)
770, r: 5.0 	-> revert, (move_up, drawer)
771, r: 5.0 	-> revert, (close, drawer)
772, r: 5.0 	-> revert, (move_up, drawer)
773, r: 5.0 	-> revert, (close, drawer)
774, r: 5.0 	-> revert, (move_up, drawer)
775, r: 5.0 	-> 

1478, r: 5.0 	-> revert, (move_up, drawer)
1479, r: 5.0 	-> revert, (close, drawer)
1480, r: 5.0 	-> revert, (move_up, drawer)
1481, r: 5.0 	-> revert, (close, drawer)
1482, r: 5.0 	-> revert, (move_up, drawer)
1483, r: 5.0 	-> revert, (close, drawer)
1484, r: 5.0 	-> revert, (move_up, drawer)
1485, r: 5.0 	-> revert, (close, drawer)
1486, r: 5.0 	-> revert, (move_up, drawer)
1487, r: 5.0 	-> revert, (close, drawer)
1488, r: 5.0 	-> revert, (move_up, drawer)
1489, r: 5.0 	-> revert, (close, drawer)
1490, r: 5.0 	-> revert, (move_up, drawer)
1491, r: 5.0 	-> revert, (close, drawer)
1492, r: 5.0 	-> revert, (move_up, drawer)
1493, r: 5.0 	-> revert, (close, drawer)
1494, r: 5.0 	-> revert, (move_up, drawer)
1495, r: 5.0 	-> revert, (close, drawer)
1496, r: 5.0 	-> revert, (move_up, drawer)
1497, r: 5.0 	-> revert, (close, drawer)
1498, r: 5.0 	-> revert, (move_up, drawer)
1499, r: 5.0 	-> revert, (close, drawer)
1500, r: 5.0 	-> revert, (move_up, drawer)
1501, r: 5.0 	-> revert, (close, 

2201, r: 5.0 	-> revert, (close, drawer)
2202, r: 5.0 	-> revert, (move_up, drawer)
2203, r: 5.0 	-> revert, (close, drawer)
2204, r: 5.0 	-> revert, (move_up, drawer)
2205, r: 5.0 	-> revert, (close, drawer)
2206, r: 5.0 	-> revert, (move_up, drawer)
2207, r: 5.0 	-> revert, (close, drawer)
2208, r: 5.0 	-> revert, (move_up, drawer)
2209, r: 5.0 	-> revert, (close, drawer)
2210, r: 5.0 	-> revert, (move_up, drawer)
2211, r: 5.0 	-> revert, (close, drawer)
2212, r: 5.0 	-> revert, (move_up, drawer)
2213, r: 5.0 	-> revert, (close, drawer)
2214, r: 5.0 	-> revert, (move_up, drawer)
2215, r: 5.0 	-> revert, (close, drawer)
2216, r: 5.0 	-> revert, (move_up, drawer)
2217, r: 5.0 	-> revert, (close, drawer)
2218, r: 5.0 	-> revert, (move_up, drawer)
2219, r: 5.0 	-> revert, (close, drawer)
2220, r: 5.0 	-> revert, (move_up, drawer)
2221, r: 5.0 	-> revert, (close, drawer)
2222, r: 5.0 	-> revert, (move_up, drawer)
2223, r: 5.0 	-> revert, (close, drawer)
2224, r: 5.0 	-> revert, (move_up, 

2921, r: 5.0 	-> revert, (close, drawer)
2922, r: 5.0 	-> revert, (move_up, drawer)
2923, r: 5.0 	-> revert, (close, drawer)
2924, r: 5.0 	-> revert, (move_up, drawer)
2925, r: 5.0 	-> revert, (close, drawer)
2926, r: 5.0 	-> revert, (move_up, drawer)
2927, r: 5.0 	-> revert, (close, drawer)
2928, r: 5.0 	-> revert, (move_up, drawer)
2929, r: 5.0 	-> revert, (close, drawer)
2930, r: 5.0 	-> revert, (move_up, drawer)
2931, r: 5.0 	-> revert, (close, drawer)
2932, r: 5.0 	-> revert, (move_up, drawer)
2933, r: 5.0 	-> revert, (close, drawer)
2934, r: 5.0 	-> revert, (move_up, drawer)
2935, r: 5.0 	-> revert, (close, drawer)
2936, r: 5.0 	-> revert, (move_up, drawer)
2937, r: 5.0 	-> revert, (close, drawer)
2938, r: 5.0 	-> revert, (move_up, drawer)
2939, r: 5.0 	-> revert, (close, drawer)
2940, r: 5.0 	-> revert, (move_up, drawer)
2941, r: 5.0 	-> revert, (close, drawer)
2942, r: 5.0 	-> revert, (move_up, drawer)
2943, r: 5.0 	-> revert, (close, drawer)
2944, r: 5.0 	-> revert, (move_up, 

...

20632, r: 5.0 	-> revert, (close, drawer)
20633, r: 5.0 	-> revert, (close, drawer)
20634, r: 5.0 	-> revert, (close, drawer)
20635, r: 5.0 	-> revert, (close, drawer)
20636, r: 5.0 	-> revert, (close, drawer)
20637, r: 5.0 	-> revert, (close, drawer)
20638, r: 5.0 	-> revert, (close, drawer)
20639, r: 5.0 	-> revert, (close, drawer)
20640, r: 5.0 	-> revert, (close, drawer)
20641, r: 5.0 	-> revert, (close, drawer)
20642, r: 5.0 	-> revert, (close, drawer)
20643, r: 5.0 	-> revert, (close, drawer)
20644, r: 5.0 	-> revert, (close, drawer)
20645, r: 5.0 	-> revert, (close, drawer)
20646, r: 5.0 	-> revert, (close, drawer)
20647, r: 5.0 	-> revert, (close, drawer)
20648, r: 5.0 	-> revert, (close, drawer)
20649, r: 5.0 	-> revert, (close, drawer)
20650, r: 5.0 	-> revert, (close, drawer)
20651, r: 5.0 	-> revert, (close, drawer)
20652, r: 5.0 	-> revert, (close, drawer)
20653, r: 5.0 	-> revert, (close, drawer)
20654, r: 5.0 	-> revert, (close, drawer)
20655, r: 5.0 	-> revert, (close, 

21373, r: 5.0 	-> revert, (close, drawer)
21374, r: 5.0 	-> revert, (close, drawer)
21375, r: 5.0 	-> revert, (close, drawer)
21376, r: 5.0 	-> revert, (close, drawer)
21377, r: 5.0 	-> revert, (close, drawer)
21378, r: 5.0 	-> revert, (close, drawer)
21379, r: 5.0 	-> revert, (close, drawer)
21380, r: 5.0 	-> revert, (close, drawer)
21381, r: 5.0 	-> revert, (close, drawer)
21382, r: 5.0 	-> revert, (close, drawer)
21383, r: 5.0 	-> revert, (close, drawer)
21384, r: 5.0 	-> revert, (close, drawer)
21385, r: 5.0 	-> revert, (close, drawer)
21386, r: 5.0 	-> revert, (close, drawer)
21387, r: 5.0 	-> revert, (close, drawer)
21388, r: 5.0 	-> revert, (close, drawer)
21389, r: 5.0 	-> revert, (close, drawer)
21390, r: 5.0 	-> revert, (close, drawer)
21391, r: 5.0 	-> revert, (close, drawer)
21392, r: 5.0 	-> revert, (close, drawer)
21393, r: 5.0 	-> revert, (close, drawer)
21394, r: 5.0 	-> revert, (close, drawer)
21395, r: 5.0 	-> revert, (close, drawer)
21396, r: 5.0 	-> revert, (close, 

22126, r: 5.0 	-> revert, (close, drawer)
22127, r: 5.0 	-> revert, (close, drawer)
22128, r: 5.0 	-> revert, (close, drawer)
22129, r: 5.0 	-> revert, (close, drawer)
22130, r: 5.0 	-> revert, (close, drawer)
22131, r: 5.0 	-> revert, (close, drawer)
22132, r: 5.0 	-> revert, (close, drawer)
22133, r: 5.0 	-> revert, (close, drawer)
22134, r: 5.0 	-> revert, (close, drawer)
22135, r: 5.0 	-> revert, (close, drawer)
22136, r: 5.0 	-> revert, (close, drawer)
22137, r: 5.0 	-> revert, (close, drawer)
22138, r: 5.0 	-> revert, (close, drawer)
22139, r: 5.0 	-> revert, (close, drawer)
22140, r: 5.0 	-> revert, (close, drawer)
22141, r: 5.0 	-> revert, (close, drawer)
22142, r: 5.0 	-> revert, (close, drawer)
22143, r: 5.0 	-> revert, (close, drawer)
22144, r: 5.0 	-> revert, (close, drawer)
22145, r: 5.0 	-> revert, (close, drawer)
22146, r: 5.0 	-> revert, (close, drawer)
22147, r: 5.0 	-> revert, (close, drawer)
22148, r: 5.0 	-> revert, (close, drawer)
22149, r: 5.0 	-> revert, (close, 

22883, r: 5.0 	-> revert, (close, drawer)
22884, r: 5.0 	-> revert, (close, drawer)
22885, r: 5.0 	-> revert, (close, drawer)
22886, r: 5.0 	-> revert, (close, drawer)
22887, r: 5.0 	-> revert, (close, drawer)
22888, r: 5.0 	-> revert, (close, drawer)
22889, r: 5.0 	-> revert, (close, drawer)
22890, r: 5.0 	-> revert, (close, drawer)
22891, r: 5.0 	-> revert, (close, drawer)
22892, r: 5.0 	-> revert, (close, drawer)
22893, r: 5.0 	-> revert, (close, drawer)
22894, r: 5.0 	-> revert, (close, drawer)
22895, r: 5.0 	-> revert, (close, drawer)
22896, r: 5.0 	-> revert, (close, drawer)
22897, r: 5.0 	-> revert, (close, drawer)
22898, r: 5.0 	-> revert, (close, drawer)
22899, r: 5.0 	-> revert, (close, drawer)
22900, r: 5.0 	-> revert, (close, drawer)
22901, r: 5.0 	-> revert, (close, drawer)
22902, r: 5.0 	-> revert, (close, drawer)
22903, r: 5.0 	-> revert, (close, drawer)
22904, r: 5.0 	-> revert, (close, drawer)
22905, r: 5.0 	-> revert, (close, drawer)
22906, r: 5.0 	-> revert, (close, 

KeyboardInterrupt: 

In [56]:
bn.policy['CM_est']

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

## Test the policy
- It trained a false policy

In [57]:
epochs, penalties, reward, = 0, 0, 0
done = False

s, s2 = t.get_start_target_scenes()
    
state = s.generate_scene_state(A, G, U, UID)
bn = MappingBayesNet(state)
bn.init_policy_simple()
steps = 0
while not done:
    bn.create_observation(t.decide(s))
    action = bn.select_action()
    s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
    steps += 1
bn.print_policy()

total: 1
0, r: False 	-> init, (put, drawer)
d: done


Policy change has no influence on the behavior 


In [58]:
bn.policy

{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init',
 'r': False,
 'permutation': 0,
 'action': {'target_object': 'drawer', 'target_action': 'put'}}

In [59]:
steps

3

Policy not trained well, error in the process. Assigning default policy

In [60]:
epochs, penalties, reward, = 0, 0, 0
done = False

s, s2 = t.get_start_target_scenes()
    
state = s.generate_scene_state(A, G, U, UID)
bn = MappingBayesNet(state)
bn.init_policy_simple()
steps = 0
while not done:
    bn.create_observation(t.decide(s))
    action = bn.select_action()
    s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
    bn.print_policy()
    steps += 1

total: 1
0, r: False 	-> init, (open, drawer)
d: done
total: 1
0, r: False 	-> init, (pick_up, cup)
d: done
total: 1
0, r: False 	-> init, (put, drawer)
d: done


In [61]:
bn.policy

{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init',
 'r': False,
 'permutation': 0,
 'action': {'target_object': 'drawer', 'target_action': 'put'}}

In [62]:
steps

3

# Troubleshooting

In [63]:
t = Tasks.CupToDrawer()
for i in range(1,101):
    epochs, penalties, reward, = 0, 0, 0
    done = False

    s, s2 = t.get_start_target_scenes()
    
    state = s.generate_scene_state(A, G, U, UID)
    bn = MappingBayesNet(state)
    bn.init_policy_simple()
    reward = 0.
    while not done:
        bn.create_observation(t.decide(s))
        action = bn.select_action()
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        bn.policy_update(reward, out=True)
        if not step_done: reward = -999

r: 5.0, diff: 1e-08
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 5.0, diff: 0.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

...

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff: -1.0
r: 4.0, diff

KeyboardInterrupt: 

In [64]:
bn.policy

{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0]]),
 'i': 'revert',
 'r': 5.0,
 'permutation': 0,
 'action': {'target_object': 'drawer', 'target_action': 'close'}}

Here it returned the diagonal matrix, which is correct.

Diagonal array policy is optimal in this setup.
Learning should reject all mutations of policy array.

In [65]:
A

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']

The criterium, that policy is correct or the best possible can be measured by steps, which are needed to make the task done.

## Single run, without changing parameters

In [66]:
t = Tasks.CupToDrawer()
policy = None
for i in range(1,2):
    epochs, penalties, reward = 0,0,0
    done = False
    
    s, s2 = t.get_start_target_scenes()
    
    state = s.generate_scene_state(A, G, U, UID)
    bn = MappingBayesNet(state)
    bn.init_policy_simple(policy)
    reward = 0.
    while not done:
        bn.create_observation(t.decide(s))
        action = bn.select_action()
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        bn.policy_step()
        if not step_done:
            reward = -999
    bn.print_policy()
    policy = bn.policy_update(reward=-len(bn.policy_history), out=True)

total: 4
0, r: False 	-> init, (open, drawer)
1, r: 0.0 	-> init, (pick_up, cup)
2, r: 0.0 	-> init, (put, drawer)
3, r: 0.0 	-> init, (put, drawer)
d: done
r: -4, diff: -4.0


## Multiple epochs
- Learning base on n of actions after every run

In [67]:
def policy_update(bn, epoch_history):
    reward = -len(bn.policy_history)
    if epoch_history != []:
        reward_dif = reward - epoch_history[-1]['r']
    else:
        reward_dif = 0.00000001

    policy = deepcopy(bn.policy) # new policy
    policy['r'] = reward
    if reward_dif >= 0:
        r1,r2 = np.random.choice(len(bn.A), 2, replace=False)
        policy['CM_est'][[r1,r2]] = policy['CM_est'][[r2,r1]]
        policy['i'] = 'forward'
    else: # Rewrites the policy from history, it has its own reward tag saved
        policy = deepcopy(epoch_history[-2])
        policy['i'] = 'revert'

    epoch_history.append(deepcopy(policy))
    return policy

A = Actions.A
G = Gestures.G
epoch_history = []
t = Tasks.CupToDrawer()
policy = {'CM_est': np.array([[0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'revert',
 'r': -52}
for i in range(1,1001):
    epochs, penalties, reward = 0,0,0
    done = False
    
    s, s2 = t.get_start_target_scenes()
    
    state = s.generate_scene_state(A, G, U, UID)
    bn = MappingBayesNet(state)
    bn.init_policy_simple(policy=policy)
    if i == 1: 
        epoch_history.append(deepcopy(bn.policy))
        epoch_history.append(deepcopy(bn.policy))
        
    reward = 0.
    n=0
    while not done:
        bn.create_observation(t.decide(s))
        action = bn.select_action()
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        bn.policy_step(reward)
        if not step_done:
            reward = -999
        n+=1
        if n>50: break
    policy = policy_update(bn, epoch_history)

In [68]:
epoch_history

[{'CM_est': array([[0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'revert',
  'r': -52},
 {'CM_est': array([[0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'revert',
  'r': -52},
 {'CM_est': array([[0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'forward',
  'r': -52,
  'action': {'target_object': 'drawer', 'target_action': 'move_up'}},
 {'CM_est': array([[1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'forward',
  'r': -52,
  'action': {'target_object': 'drawer', 'target_action': 'move_up'

In [69]:
policy

{'CM_est': array([[0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'revert',
 'r': -4,
 'action': {'target_object': 'drawer', 'target_action': 'put'}}

Algorithm won't learn.
I will try to reduce actions, try to simplify and see what is going on:

In [70]:
import importlib
import model_noK_evensimplified
importlib.reload(model_noK_evensimplified)

from model_noK_evensimplified import MappingBayesNet_EvenSimplified

In [71]:
def policy_update(bn, epoch_history):
    reward = -len(bn.policy_history)
    if epoch_history != []:
        reward_dif = reward - epoch_history[-1]['r']
    else:
        reward_dif = 0.00000001

    policy = deepcopy(bn.policy) # new policy
    policy['r'] = reward
    if reward_dif >= 0:
        r1,r2 = np.random.choice(len(bn.A), 2, replace=False)
        policy['CM_est'][[r1,r2]] = policy['CM_est'][[r2,r1]]
        policy['i'] = 'forward'
    else: # Rewrites the policy from history, it has its own reward tag saved
        policy = deepcopy(epoch_history[-2])
        policy['i'] = 'revert'

    epoch_history.append(deepcopy(policy))
    return policy

A = ['pick_up', 'put', 'open']
G = ['swipe_up', 'swipe_down','open']

epoch_history = []
t = Tasks.CupToDrawer()
policy = {'CM_est': np.array([[0, 1, 0],
                              [1, 0, 0],
                              [0, 0, 1]]), 'i': 'revert', 'r': -52}
for i in range(1,101):
    s, s2 = t.get_start_target_scenes()
    
    state = s.generate_scene_state(A, G, U, UID)
    bn = MappingBayesNet_EvenSimplified(state)
    bn.init_policy_simple(policy=policy)
    if i == 1: epoch_history.append(deepcopy(bn.policy))
    reward = 0.
    n=0
    done = False
    while not done:
        bn.create_observation(t.decide(s))
        action = bn.select_action()
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        bn.policy_step(reward)
        if not step_done:
            reward = -999
        n+=1
        if n>50: break
    policy = policy_update(bn, epoch_history)


In [72]:
bn.print_policy()

total: 4
0, r: -52 	-> revert, (open, drawer)
1, r: 5.0 	-> revert, (pick_up, cup)
2, r: 5.0 	-> revert, (put, drawer)
3, r: True 	-> revert, (put, drawer)
d: done


In [73]:
bn.policy

{'CM_est': array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]),
 'i': 'revert',
 'r': True,
 'action': {'target_object': 'drawer', 'target_action': 'put'}}

In [74]:
bn.observation

{'focus_point': array([[3.22247498, 0.09160486, 0.040837  ]]),
 'gesture_vec': array([0.02718953, 0.84395905, 0.12885142])}

In [75]:
action

{'target_object': 'drawer', 'target_action': 'put'}